In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn


In [3]:
!git clone https://github.com/zeinashaarawy/DataOrbit

Cloning into 'DataOrbit'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 22 (delta 3), reused 19 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 24.34 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [5]:
%cd DataOrbit

/content/DataOrbit


In [6]:
!git checkout Zeina

Branch 'Zeina' set up to track remote branch 'Zeina' from 'origin'.
Switched to a new branch 'Zeina'


In [7]:
!git pull origin main

From https://github.com/zeinashaarawy/DataOrbit
 * branch            main       -> FETCH_HEAD
Updating 82d7ce4..ccce9b9
Fast-forward
 README.md                        |      0
 data/Train_Beneficiarydata.csv   | 138557 ++++++++++
 data/Train_Inpatientdata.csv     |  40475 +++
 data/Train_Outpatientdata.csv    | 517738 ++++++++++++++++++++++++++++++++++++
 data/Train_labels.csv            |   5411 +
 file.txt                         |      1 -
 notebooks/data_exploration.ipynb |     46 +
 7 files changed, 702227 insertions(+), 1 deletion(-)
 create mode 100644 README.md
 create mode 100644 data/Train_Beneficiarydata.csv
 create mode 100644 data/Train_Inpatientdata.csv
 create mode 100644 data/Train_Outpatientdata.csv
 create mode 100644 data/Train_labels.csv
 delete mode 100644 file.txt
 create mode 100644 notebooks/data_exploration.ipynb


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
beneficiaries = pd.read_csv('data/Train_Beneficiarydata.csv')
inpatient = pd.read_csv('data/Train_Inpatientdata.csv')
outpatient = pd.read_csv('data/Train_Outpatientdata.csv')
labels = pd.read_csv('data/Train_labels.csv')


In [11]:
print(beneficiaries.shape)
print(inpatient.shape)
print(outpatient.shape)
print(labels.shape)

beneficiaries.head()


(138556, 25)
(40474, 30)
(517737, 27)
(5410, 2)


,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


In [13]:
print("Missing values in Beneficiaries:")
print(beneficiaries.isnull().sum())

print("\nMissing values in Inpatient:")
print(inpatient.isnull().sum())

print("\nMissing values in Outpatient:")
print(outpatient.isnull().sum())

print("\nMissing values in Labels:")
print(labels.isnull().sum())


Missing values in Beneficiaries:
BeneID                                  0
DOB                                     0
DOD                                137135
Gender                                  0
Race                                    0
RenalDiseaseIndicator                   0
State                                   0
County                                  0
NoOfMonths_PartACov                     0
NoOfMonths_PartBCov                     0
ChronicCond_Alzheimer                   0
ChronicCond_Heartfailure                0
ChronicCond_KidneyDisease               0
ChronicCond_Cancer                      0
ChronicCond_ObstrPulmonary              0
ChronicCond_Depression                  0
ChronicCond_Diabetes                    0
ChronicCond_IschemicHeart               0
ChronicCond_Osteoporasis                0
ChronicCond_rheumatoidarthritis         0
ChronicCond_stroke                      0
IPAnnualReimbursementAmt                0
IPAnnualDeductibleAmt                   0
O

In [15]:
def clean_column_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

In [16]:
beneficiaries = clean_column_names(beneficiaries)
inpatient = clean_column_names(inpatient)
outpatient = clean_column_names(outpatient)
labels = clean_column_names(labels)


Created is_deceased. When date of date is missing, patient is deceased. (1 is deceased and 0 is not)

In [17]:
beneficiaries['is_deceased'] = beneficiaries['dod'].notna().astype(int)

Dropped Columns with very high missing values

In [18]:
inpatient_drop_cols = [
    'operatingphysician', 'otherphysician',
    'clmdiagnosiscode_2','clmdiagnosiscode_3','clmdiagnosiscode_4','clmdiagnosiscode_5',
    'clmdiagnosiscode_6','clmdiagnosiscode_7','clmdiagnosiscode_8','clmdiagnosiscode_9','clmdiagnosiscode_10',
    'clmprocedurecode_1','clmprocedurecode_2','clmprocedurecode_3','clmprocedurecode_4','clmprocedurecode_5','clmprocedurecode_6'
]

In [19]:
inpatient_cleaned = inpatient.drop(columns=inpatient_drop_cols)

Filled in small missing values

In [20]:
inpatient_cleaned['attendingphysician'] = inpatient_cleaned['attendingphysician'].fillna('Unknown')
inpatient_cleaned['deductibleamtpaid'] = inpatient_cleaned['deductibleamtpaid'].fillna(0)

In [21]:
outpatient_drop_cols = [
    'operatingphysician', 'otherphysician',
    'clmdiagnosiscode_2','clmdiagnosiscode_3','clmdiagnosiscode_4','clmdiagnosiscode_5',
    'clmdiagnosiscode_6','clmdiagnosiscode_7','clmdiagnosiscode_8','clmdiagnosiscode_9','clmdiagnosiscode_10',
    'clmprocedurecode_1','clmprocedurecode_2','clmprocedurecode_3','clmprocedurecode_4','clmprocedurecode_5','clmprocedurecode_6',
    'clmadmitdiagnosiscode'
]


In [22]:
outpatient_cleaned = outpatient.drop(columns=outpatient_drop_cols)

In [24]:
outpatient_cleaned['attendingphysician'] = outpatient_cleaned['attendingphysician'].fillna('Unknown')

Merge with beneficiary on beneid

In [25]:
inpatient_merged = inpatient_cleaned.merge(beneficiaries, on='beneid', how='left')
outpatient_merged = outpatient_cleaned.merge(beneficiaries, on='beneid', how='left')


Aggregate to combine all claims of provider into one row

In [26]:
provider_inpatient = inpatient_merged.groupby('provider').agg({
    'inscclaimamtreimbursed': ['sum','mean','max'],
    'deductibleamtpaid': ['sum','mean'],
    'beneid': 'nunique',
    'is_deceased': 'sum'  # number of deceased patients per provider
}).reset_index()

In [27]:
provider_outpatient = outpatient_merged.groupby('provider').agg({
    'inscclaimamtreimbursed': ['sum','mean','max'],
    'deductibleamtpaid': ['sum','mean'],
    'beneid': 'nunique',
    'is_deceased': 'sum'
}).reset_index()

Multiindex Columns


In [28]:
provider_inpatient.columns = [
    'provider', 'ip_claim_sum', 'ip_claim_mean', 'ip_claim_max',
    'ip_deductible_sum', 'ip_deductible_mean', 'ip_unique_patients', 'ip_deceased_patients'
]

provider_outpatient.columns = [
    'provider', 'op_claim_sum', 'op_claim_mean', 'op_claim_max',
    'op_deductible_sum', 'op_deductible_mean', 'op_unique_patients', 'op_deceased_patients'
]

In [29]:
print("Unique providers (inpatient):", inpatient_merged['provider'].nunique())
print("Unique providers (outpatient):", outpatient_merged['provider'].nunique())

print(provider_inpatient.head())
print(provider_outpatient.head())

Unique providers (inpatient): 2092
Unique providers (outpatient): 5012
   provider  ip_claim_sum  ip_claim_mean  ip_claim_max  ip_deductible_sum  \
0  PRV51001         97000   19400.000000         42000             5340.0   
1  PRV51003        573000    9241.935484         57000            66216.0   
2  PRV51007         19000    6333.333333         10000             3204.0   
3  PRV51008         25000   12500.000000         21000             2136.0   
4  PRV51011          5000    5000.000000          5000             1068.0   

   ip_deductible_mean  ip_unique_patients  ip_deceased_patients  
0              1068.0                   5                     0  
1              1068.0                  53                     1  
2              1068.0                   3                     0  
3              1068.0                   2                     0  
4              1068.0                   1                     0  
   provider  op_claim_sum  op_claim_mean  op_claim_max  op_deductible_

Merge to get single provider-level dataset

In [30]:
provider_full = provider_inpatient.merge(provider_outpatient, on='provider', how='outer').fillna(0)

Merge to add target values

In [32]:
provider_full = provider_full.merge(labels, on='provider', how='left')

Fix duplicate label columns


In [36]:
provider_full['potentialfraud'] = provider_full['potentialfraud_x'].combine_first(provider_full['potentialfraud_y'])
provider_full = provider_full.drop(columns=['potentialfraud_x','potentialfraud_y'])

Add derived features

In [41]:
#Total claims (sum of inpatient + outpatient)
provider_full['total_claim_sum'] = provider_full['ip_claim_sum'] + provider_full['op_claim_sum']

#Average claim amount across all claims
provider_full['total_claim_mean'] = (provider_full['ip_claim_sum'] + provider_full['op_claim_sum']) / \
                                    (provider_full['ip_unique_patients'] + provider_full['op_unique_patients'] + 1e-6)

#Total deductible paid
provider_full['total_deductible_sum'] = provider_full['ip_deductible_sum'] + provider_full['op_deductible_sum']

#Average deductible per patient
provider_full['total_deductible_mean'] = (provider_full['ip_deductible_sum'] + provider_full['op_deductible_sum']) / \
                                         (provider_full['ip_unique_patients'] + provider_full['op_unique_patients'] + 1e-6)

#Total unique patients
provider_full['total_unique_patients'] = provider_full['ip_unique_patients'] + provider_full['op_unique_patients']

#Total deceased patients
provider_full['total_deceased_patients'] = provider_full['ip_deceased_patients'] + provider_full['op_deceased_patients']

#Ratio of deceased patients
provider_full['deceased_patient_ratio'] = provider_full['total_deceased_patients'] / \
                                          (provider_full['total_unique_patients'] + 1e-6)

#Claim per patient ratio
provider_full['claim_per_patient'] = provider_full['total_claim_sum'] / \
                                     (provider_full['total_unique_patients'] + 1e-6)

#Inpatient / Outpatient claim ratio
provider_full['ip_op_claim_ratio'] = provider_full['ip_claim_sum'] / (provider_full['op_claim_sum'] + 1e-6)

#Max claim / mean claim ratio (inpatient)
provider_full['ip_max_mean_ratio'] = provider_full['ip_claim_max'] / (provider_full['ip_claim_mean'] + 1e-6)

# 3. Chronic condition ratio (example using total patients with chronic conditions)
# Assuming you have aggregated chronic conditions from beneficiary data
# Example: provider_full['chronic_patients'] = number of patients with ≥1 chronic condition
provider_full['chronic_cond_ratio'] = provider_full.get('chronic_patients', 0) / \
                                     (provider_full['total_unique_patients'] + 1e-6)

#Average claim per deceased patient
provider_full['avg_claim_per_deceased'] = provider_full['total_claim_sum'] / \
                                         (provider_full['total_deceased_patients'] + 1e-6)


In [42]:
print(provider_full.isna().sum())  # Make sure no NaNs remain
print(provider_full.dtypes)

provider                   0
ip_claim_sum               0
ip_claim_mean              0
ip_claim_max               0
ip_deductible_sum          0
ip_deductible_mean         0
ip_unique_patients         0
ip_deceased_patients       0
op_claim_sum               0
op_claim_mean              0
op_claim_max               0
op_deductible_sum          0
op_deductible_mean         0
op_unique_patients         0
op_deceased_patients       0
total_claim_sum            0
total_claim_mean           0
total_deductible_sum       0
total_deductible_mean      0
total_unique_patients      0
total_deceased_patients    0
deceased_patient_ratio     0
claim_per_patient          0
potentialfraud             0
ip_op_claim_ratio          0
ip_max_mean_ratio          0
chronic_cond_ratio         0
avg_claim_per_deceased     0
dtype: int64
provider                    object
ip_claim_sum               float64
ip_claim_mean              float64
ip_claim_max               float64
ip_deductible_sum          float64


In [43]:
provider_full.to_csv('data/provider_final_features.csv', index=False)

In [44]:
print(provider_full.head())

   provider  ip_claim_sum  ip_claim_mean  ip_claim_max  ip_deductible_sum  \
0  PRV51001       97000.0   19400.000000       42000.0             5340.0   
1  PRV51003      573000.0    9241.935484       57000.0            66216.0   
2  PRV51004           0.0       0.000000           0.0                0.0   
3  PRV51005           0.0       0.000000           0.0                0.0   
4  PRV51007       19000.0    6333.333333       10000.0             3204.0   

   ip_deductible_mean  ip_unique_patients  ip_deceased_patients  op_claim_sum  \
0              1068.0                 5.0                   0.0        7640.0   
1              1068.0                53.0                   1.0       32670.0   
2                 0.0                 0.0                   0.0       52170.0   
3                 0.0                 0.0                   0.0      280910.0   
4              1068.0                 3.0                   0.0       14710.0   

   op_claim_mean  ...  total_deductible_mean  tota